In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, roc_curve, auc
import seaborn as sns
import random
from glob import glob
import os


# Data Loading 

In [ ]:
#global declarations
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

In [ ]:
data_dir = r"/kaggle/input/cell-images-for-detecting-malaria/"
target_classes = os.listdir(data_dir)
print(target_classes)


In [ ]:
infected_images     = glob(os.path.join(data_dir, target_classes[0])+"/*.png")
uninfected_images   = glob(os.path.join(data_dir, target_classes[1])+"/*.png")

data = []

for image_path in (infected_images + uninfected_images): 
     
        label = target_classes[0] if (target_classes[0] in image_path) else target_classes[1] 
        data.append((image_path, label))
      

data_df = pd.DataFrame(data, columns=["image_path", "label"])
data_df.head()

In [ ]:
# Get the counts for each class
sample_count = data_df['label'].value_counts()
print(sample_count)

# Plot the results 
plt.figure(figsize=(10,8))
sns.barplot(x=sample_count.index, y= sample_count.values)
plt.title('Number of cases', fontsize=14)
plt.xlabel('Case type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(range(len(sample_count.index)), ['Parasitized(1)', 'Uninfected(0)'])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
fig.suptitle('Parasitized cells', fontsize=24)

for ind, img_src in enumerate(infected_images[:30]):
    plt.subplot(3, 10, ind+1)
    img = plt.imread(img_src)
    plt.axis('off')
    plt.imshow(img)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
fig.suptitle('Uninfected cells', fontsize=24)

for ind, img_src in enumerate(uninfected_images[:30]):
    plt.subplot(3, 10, ind+1)
    img = plt.imread(img_src)
    plt.axis('off')
    plt.imshow(img)

# Preparing data for training and validation

In [ ]:
train_df, test_df = train_test_split(data_df, test_size=0.30, random_state=42)
train_df, validate_df = train_test_split(train_df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
total_test = test_df.shape[0]
batch_size=16
print(f"Total training samples : {len(train_df)}\nTotal validation samples : {len(validate_df)}\nTotal Test samples : {len(test_df)}")

In [ ]:
# Applying Augmentation to genearate images with different angles, shift, flips etc.
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

validation_datagen = ImageDataGenerator(rescale=1./255)


# VGG 19

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import  Adam
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
x_train_vgg =  train_datagen.flow_from_dataframe(dataframe = train_df,  x_col='image_path', y_col='label',  class_mode='categorical',target_size=(224,224), shuffle=False, batch_size=10, seed=10)
x_val_vgg = validation_datagen.flow_from_dataframe(dataframe = validate_df,  x_col='image_path', y_col='label',class_mode='categorical',  target_size=(224,224), shuffle=False, batch_size=10, seed=10)

vgg19_model = VGG19(input_shape=(224,224,3), weights='imagenet',include_top=False)
model=Sequential()
model.add(vgg19_model)
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer=Adam(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

model_history = model.fit(
x_train_vgg,
steps_per_epoch=100,
validation_data=x_val_vgg,
validation_steps=100, 
epochs = 10)
    
    

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (30, 10))
ax = ax.ravel()

for i, metric in enumerate(["accuracy", "loss"]):
    ax[i].plot(model_history.history[metric])
    ax[i].plot(model_history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("Epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])

In [ ]:
model.evaluate(x_val_vgg)

# validation loss = 0.69
# validation Accuracy = ~50%

*Performs poor, maybe the model is too complex*

# VGG16

In [ ]:
# Using the flow_from_dataframe method to generate batches of training data from a DataFrame
x_train_vgg16 = train_datagen.flow_from_dataframe(
    dataframe=train_df,                 
    x_col='image_path',                 
    y_col='label',                      
    class_mode='binary',                
    target_size=IMAGE_SIZE,             
    shuffle=False,                     
    batch_size=10,                      
    seed=10                             
)

# Using the flow_from_dataframe method to generate batches of validation data from a DataFrame
x_val_vgg16 = validation_datagen.flow_from_dataframe(
    dataframe=validate_df,              
    x_col='image_path',                 
    y_col='label',                      
    class_mode='binary',               
    target_size=IMAGE_SIZE,             
    shuffle=False,                     
    batch_size=10,                      
    seed=10                            
)


In [ ]:
 # Import VGG16 model and set its input shape to (128,128,3)
vgg16 = VGG16(input_shape=(128,128,3), weights='imagenet', include_top=False) 
  

# Freeze all the layers in the VGG16 model so that their weights will not be updated during training  
for layer in vgg16.layers:
    layer.trainable = False

# Add classification head
x = Flatten()(vgg16.output)
x = Dense(4096)(x)
prediction = Dense(1, activation='sigmoid')(x)

vgg16_model = Model(inputs=vgg16.input, outputs=prediction)
vgg16_model.summary()

In [ ]:
# Compile the VGG16 model with Adam optimizer, binary crossentropy loss, and accuracy metric
vgg16_model.compile(optimizer='adam',
              loss=['binary_crossentropy'],
              metrics=["accuracy"])

history_vgg16 = vgg16_model.fit(x_train_vgg16,
                    epochs =30,
                    steps_per_epoch=100,
                    validation_data = x_val_vgg16,
                    validation_steps=100, 
                    batch_size=10)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (30, 10))
ax = ax.ravel()

for i, metric in enumerate(["accuracy", "loss"]):
    ax[i].plot(history_vgg16.history[metric])
    ax[i].plot(history_vgg16.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("Epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])

In [ ]:
vgg16_model.evaluate(x_val_vgg16)   # evaluation on validation set

# Validation loss = 0.26

# Validation Accuracy = ~ 90.6 %

*The VGG19 model has lower accuracy compared to the VGG16 model, possibly due to its deeper architecture which may lead to overfitting and slower training. On the other hand, the VGG16 model has fewer layers and is pretrained on a larger dataset, resulting in better feature extraction capabilities and better generalization performance. Therefore, we have an insight here that it is not always the case that a deeper model will perform better, as it depends on the specific dataset.*

# CUSTOM CNN ARCHITECTURE

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,  
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=16
)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=16
)



total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
total_test = test_df.shape[0]
batch_size=16

print(f"Training data : {total_train}\nvalidation data : {total_validate}\nTest data : {total_test}")

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import  RMSprop


# Define a sequential model
model = Sequential()

# Add convolutional layer with 32 filters, kernel size of (3,3), ReLU activation, and input shape
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
# Add max pooling layer with pool size of (2,2)
model.add(MaxPooling2D(pool_size=(2, 2)))                                                                                   # CONV BLOCK 1
# Add batch normalization layer
model.add(BatchNormalization())

# Add convolutional layer with 32 filters, kernel size of (3,3), ReLU activation
model.add(Conv2D(32, (3, 3), activation='relu'))
# Add max pooling layer with pool size of (2,2)
model.add(MaxPooling2D(pool_size=(2, 2)))
# Add batch normalization layer                                                                                               # CONV BLOCK 2
model.add(BatchNormalization())
# Add dropout layer with a rate of 0.5
model.add(Dropout(0.5))

# Add convolutional layer with 64 filters, kernel size of (3,3), ReLU activation
model.add(Conv2D(64, (3, 3), activation='relu'))
# Add batch normalization layer
model.add(BatchNormalization())
# Add max pooling layer with pool size of (2,2)                                                                              # CONV BLOCK 3
model.add(MaxPooling2D(pool_size=(2, 2)))
# Add dropout layer with a rate of 0.25
model.add(Dropout(0.25))

# Add flatten layer
model.add(Flatten())

# Add dense layer with 128 units and ReLU activation
model.add(Dense(128, activation='relu'))
# Add dropout layer with a rate of 0.5
model.add(Dropout(0.5))                                                                                                     # CLASSIFICATION HEAD
# Add dense layer with 2 units and softmax activation
model.add(Dense(2, activation='softmax')) 

# Define optimizer as RMSprop with learning rate of 0.0001
optimizer = RMSprop(learning_rate=0.0001)

# Compile model with categorical crossentropy loss and accuracy metric
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Print model summary
model.summary()


In [ ]:
# Fit the model and train for 30 epochs
epochs = 30
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (30, 10))
ax = ax.ravel()

for i, metric in enumerate(["accuracy", "loss"]):
    ax[i].plot(history.history[metric])
    ax[i].plot(history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("Epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])

In [ ]:
model.evaluate(validation_generator)

# Validation loss = 0.185

# Validation Accuracy = ~ 95 %

*Supporting our previous insight that for our dataset, a simpler architecture works better than a complex one as vgg19. When compared to vgg16 which performed signifcantly better than vgg19, The custom CNN, despite its simpler architecture, has achieved a high validation accuracy of 95%, which is comparable to the VGG16 model's accuracy of 90%.*

*By looking at the loss & accuracy curves, we can observe some sudden steeps. This indicates that there is some noise out there which hinders the model's convergance at regular intervals. Now, we look forward to tune the performance of this custom cnn model and get smooth training curves with hopefully a lesser loss and higher accuracy*

# Tuning the CNN

In [ ]:
batch_size=16
x_train_tuned_cnn =  train_datagen.flow_from_dataframe(
dataframe = train_df,
x_col='image_path',
y_col='label',
class_mode='categorical',
target_size=IMAGE_SIZE,
shuffle=False,
batch_size=batch_size,
seed=10)



x_val_tuned_cnn = validation_datagen.flow_from_dataframe(
dataframe = validate_df,
x_col='image_path',
y_col='label',
class_mode='categorical',
target_size=IMAGE_SIZE,
shuffle=False,
batch_size=batch_size,
seed=10)

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
total_test = test_df.shape[0]
batch_size=16

print(f"Training data : {total_train}\nvalidation data : {total_validate}\nTest data : {total_test}")


In [ ]:
# Define a sequential model
tuned_model = Sequential()

# Add the first convolutional layer with 16 filters and a 3x3 kernel
tuned_model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
# Add max pooling layer with a 2x2 pool size
tuned_model.add(MaxPooling2D(pool_size=(2, 2)))                                                                         # CONV BLOCK 1
# Add dropout layer with a rate of 0.2
tuned_model.add(Dropout(0.2))

# Add second convolutional layer with 32 filters and a 3x3 kernel
tuned_model.add(Conv2D(32, (3, 3), activation='relu'))
# Add max pooling layer with a 2x2 pool size                                                                            # CONV BLOCK 2
tuned_model.add(MaxPooling2D(pool_size=(2, 2)))
# Add dropout layer with a rate of 0.25
tuned_model.add(Dropout(0.25))

# Add third convolutional layer with 64 filters and a 3x3 kernel
tuned_model.add(Conv2D(64, (3, 3), activation='relu'))
# Add max pooling layer with a 2x2 pool size                                                                            # CONV BLOCK 3
tuned_model.add(MaxPooling2D(pool_size=(2, 2)))
# Add dropout layer with a rate of 0.25
tuned_model.add(Dropout(0.25))

# Add flatten layer to convert the output from 2D to 1D
tuned_model.add(Flatten())

# Add a fully connected layer with 1024 units and ReLU activation
tuned_model.add(Dense(1024, activation='relu'))                                                                         # CLASSIFICATION HEAD
# Add batch normalization layer
tuned_model.add(BatchNormalization())
# Add dropout layer with a rate of 0.5
tuned_model.add(Dropout(0.5))

# Add a fully connected layer with 512 units and ReLU activation
tuned_model.add(Dense(512, activation='relu'))
# Add batch normalization layer
tuned_model.add(BatchNormalization())
# Add dropout layer with a rate of 0.5
tuned_model.add(Dropout(0.5))

# Add a fully connected layer with 128 units and ReLU activation
tuned_model.add(Dense(128, activation='relu'))
# Add batch normalization layer
tuned_model.add(BatchNormalization())
# Add dropout layer with a rate of 0.5
tuned_model.add(Dropout(0.5))

# Add output layer with 2 units and softmax activation for binary classification
tuned_model.add(Dense(2, activation='softmax')) 

# Define Adam optimizer with a learning rate of 0.0001
optimizer = Adam(learning_rate=0.0001)

# Compile the model with categorical crossentropy loss and accuracy metric
tuned_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Print the model summary
tuned_model.summary()

In [ ]:
tuned_model.compile(loss = "categorical_crossentropy" , optimizer = "adam" , metrics = ["accuracy"])
tuned_model_history = tuned_model.fit_generator(
    x_train_tuned_cnn, 
    epochs=30,
    validation_data=x_val_tuned_cnn,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size
)


In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (30, 10))
ax = ax.ravel()

for i, metric in enumerate(["accuracy", "loss"]):
    ax[i].plot(tuned_model_history.history[metric])
    ax[i].plot(tuned_model_history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("Epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])

*Observastion : The Accuracy and loss curves are now way smoother than the earlier model*

In [ ]:
tuned_model.evaluate(x_val_tuned_cnn)

# Validation loss = 0.13

# Validation Accuracy = ~ 96 %

- Induced the feature pyramid culture by decreasing the no. of filters in the initial convolutional layers and added more dense layers in the classification head, resulting in a more powerful model with increased capacity for feature extraction and classification.
- Reduced the dropout rate in the initial layers to retain more information from the input, allowing the model to learn more effectively from the data.
- Changed the optimizer from RMSprop to Adam, which is known to perform well on a wide range of deep learning tasks, resulting in better optimization and faster convergence of the training process.

# Evaluation using various metrics on test set

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    x_col='image_path',
    y_col='label',
    class_mode='categorical',
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
# make predictions using the model
y_pred = tuned_model.predict_generator(test_generator, steps=test_generator.samples)

# get the true labels
y_true = test_generator.classes

# get the class indices
class_indices = test_generator.class_indices

# get the class names
class_names = list(class_indices.keys())

# get the predicted labels
y_pred = np.argmax(y_pred, axis=1)

# create the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# print the confusion matrix
print(cm)

# plot the confusion matrix
# create a heatmap of the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=class_indices.keys(), yticklabels=class_indices.keys())
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()



## Cost matrix Readings

TP = 3901, TN = 4096, FP = 116, FN = 245

In [ ]:
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# calculate precision, recall, f1 score, and support
precision, recall, f1_score, support = precision_recall_fscore_support(y_true, y_pred)

# plot precision, recall, and f1 score
fig, ax = plt.subplots()
x_labels = list(class_indices.keys())
x_pos = np.arange(len(x_labels))
ax.bar(x_pos - 0.2, precision, width=0.2, label='Precision')
ax.bar(x_pos, recall, width=0.2, label='Recall')
ax.bar(x_pos + 0.2, f1_score, width=0.2, label='F1 Score')
ax.set_xticks(x_pos)
ax.set_xticklabels(x_labels)
ax.set_ylim([0, 1])
ax.set_xlabel('Class')
ax.set_ylabel('Score')
ax.set_title('Precision, Recall, and F1 Score')
ax.legend()
plt.show()

- Precision: It is the ratio of true positives to the sum of true positives and false positives. In other words, it measures the proportion of positive predictions that were actually correct. For the class "Parasitized", the precision is 0.97, which means that 97% of the images predicted as "Parasitized" were actually Parasitized. Similarly, for the class "Uninfected", the precision is 0.94, which means that 94% of the images predicted as "Uninfected" were actually Uninfected.
- Recall: It is the ratio of true positives to the sum of true positives and false negatives. In other words, it measures the proportion of actual positives that were correctly identified. For the class "Parasitized", the recall is 0.94, which means that 94% of the actual Parasitized images were correctly identified as Parasitized. Similarly, for the class "Uninfected", the recall is 0.97, which means that 97% of the actual Uninfected images were correctly identified as Uninfected.
- F1-score: It is the harmonic mean of precision and recall, and provides a single score that balances both these metrics. For both the classes, the F1-score is 0.96, which means that the model is performing equally well on both the classes.
- Support: It is the number of samples in each class.

In summary, the model is performing well with an overall accuracy of 0.96, and good precision, recall and F1-score for both the classes.



In [ ]:
# calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred)

# calculate the AUC ROC
auc_score = auc(fpr, tpr)

# plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")

plt.show()


- The area under the receiver operating characteristic (ROC) curve for the model is 0.96, indicating that the model has good predictive performance.